In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import xlsxwriter
import requests
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
def chunks(lst, n):
    
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [4]:
def final_list(value):
    
    stock = 'AAPL' #holder for stock name
    my_columns = ['Ticker','Price','One Year Return', 'Number Of Shares to Buy']
    global symbol_groups
    symbol_groups = list(chunks(stocks['Ticker'], 100))
    global symbol_strings
    symbol_strings = []
    
    for i in range(0, len(symbol_groups)):
        symbol_strings.append(','.join(symbol_groups[i]))
    
    df = pd.DataFrame(columns=my_columns)
    for name in symbol_strings:
        
        batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={name}&token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(batch_api_url).json()
        
        
        for quote in name.split(','):
            try:
                df = df.append(
                    pd.Series(
                        [
                            quote,
                            data[quote]['quote']['latestPrice'],
                            data[quote]['stats']['year1ChangePercent'],
                            'N/A'
                            ],
                        index=my_columns
                        ),
                    ignore_index=True
                    )
            except:
                continue
    return df
        # print(data.status_code)
        # print(name)

data = final_list(stocks)
data

,Ticker,Price,One Year Return,Number Of Shares to Buy
0,A,158.93,0.01049,N/A
1,AAL,14.68,-0.262384,N/A
2,AAP,157.55,-0.357546,N/A
3,AAPL,157.34,-0.06034,N/A
4,ABBV,160.94,0.402932,N/A
...,...,...,...,...
496,YUM,128.02,0.018154,N/A
497,ZBH,119.44,-0.09361,N/A
498,ZBRA,282.11,-0.56293,N/A
499,ZION,53.78,-0.224651,N/A


In [5]:
data.sort_values(by='One Year Return', ascending=False, inplace=True)
data = data[:50]
data.reset_index(inplace=True, drop=True)
data

,Ticker,Price,One Year Return,Number Of Shares to Buy
0,LB,79.96,2.315811,N/A
1,OXY,71.25,1.298912,N/A
2,MPC,131.80,1.032039,N/A
3,VLO,142.19,1.003793,N/A
4,FTI,12.65,0.921337,N/A
5,MRO,31.38,0.884272,N/A
6,XOM,118.87,0.883492,N/A
7,HES,146.60,0.821245,N/A
8,COP,129.40,0.80029,N/A
9,HRB,42.86,0.783789,N/A


In [6]:
def portfolio_input():
    
    global portfolio_size
    
    portfolio_size = 10000000
    
    # try:
    #     float(portfolio_size)
    # except ValueError:
    #     print('That is a wrong Input \nPlease Try again')
    #     portfolio_size = input('Enter your portfolio value:')
        
portfolio_input()
print(portfolio_size)

10000000


In [7]:
position_size = float(portfolio_size)/len(data.index)
data['Price'] = pd.to_numeric(data['Price'], downcast='float')

for i in range(0, len(data.index)):
    data.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size/data.loc[i, 'Price'])
    
data

,Ticker,Price,One Year Return,Number Of Shares to Buy
0,LB,79.959999,2.315811,2501
1,OXY,71.250000,1.298912,2807
2,MPC,131.800003,1.032039,1517
3,VLO,142.190002,1.003793,1406
4,FTI,12.650000,0.921337,15810
5,MRO,31.379999,0.884272,6373
6,XOM,118.870003,0.883492,1682
7,HES,146.600006,0.821245,1364
8,COP,129.399994,0.80029,1545
9,HRB,42.860001,0.783789,4666


In [8]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of shares to buy',
    'one-year price return',
    'one-year return percentile',
    'six-month price return',
    'six-month return percentile',
    'three-month price return',
    'three-month return percentile',
    'one-month price return',
    'one-month return percentile',
    'hqm_score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
hqm_dataframe

,Ticker,Price,Number of shares to buy,one-year price return,one-year return percentile,six-month price return,six-month return percentile,three-month price return,three-month return percentile,one-month price return,one-month return percentile,hqm_score


In [9]:
for name in symbol_strings:
    
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={name}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    # print(data.status_code)
    
    for symbol in name.split(','):
        try:
            hqm_dataframe = hqm_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        data[symbol]['quote']['latestPrice'],
                        'N/A',
                        data[symbol]['stats']['year1ChangePercent'],
                        'N/A',
                        data[symbol]['stats']['month6ChangePercent'],
                        'N/A',
                        data[symbol]['stats']['month3ChangePercent'],
                        'N/A',
                        data[symbol]['stats']['month1ChangePercent'],
                        'N/A',
                        'N/A'
                        ],
                    index=hqm_columns
                    ),
                ignore_index = True
                )
        except:
            continue
          
hqm_dataframe

,Ticker,Price,Number of shares to buy,one-year price return,one-year return percentile,six-month price return,six-month return percentile,three-month price return,three-month return percentile,one-month price return,one-month return percentile,hqm_score
0,A,157.59,N/A,0.010338,N/A,0.243408,N/A,0.175555,N/A,0.197939,N/A,N/A
1,AAL,14.68,N/A,-0.267686,N/A,-0.14114,N/A,0.043814,N/A,0.053156,N/A,N/A
2,AAP,154.86,N/A,-0.352359,N/A,-0.165201,N/A,-0.24115,N/A,-0.127095,N/A,N/A
3,AAPL,152.55,N/A,-0.060814,N/A,0.05924,N/A,-0.099102,N/A,0.027899,N/A,N/A
4,ABBV,162.97,N/A,0.412061,N/A,0.101334,N/A,0.160848,N/A,0.086984,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,127.45,N/A,0.018128,N/A,0.123818,N/A,0.110198,N/A,0.145274,N/A,N/A
497,ZBH,118.35,N/A,-0.093202,N/A,-0.019494,N/A,0.082719,N/A,0.100891,N/A,N/A
498,ZBRA,273.17,N/A,-0.574901,N/A,-0.19716,N/A,-0.151993,N/A,0.030305,N/A,N/A
499,ZION,53.69,N/A,-0.222786,N/A,-0.037771,N/A,-0.078719,N/A,0.086925,N/A,N/A


In [10]:
time_periods = ['one-year','six-month','three-month','one-month']

for column in ['one-year price return', 'six-month price return', 'three-month price return', 'one-month price return']:
    hqm_dataframe[column].fillna(hqm_dataframe[column].mean(), inplace = True)

for rows in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} price return'
        percentile_col = f'{time_period} return percentile'
        hqm_dataframe[f'{time_period} price return'] =  pd.to_numeric(hqm_dataframe[f'{time_period} price return'])
        hqm_dataframe.loc[rows, f'{time_period} return percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} price return'], hqm_dataframe.loc[rows, f'{time_period} price return'])
        
hqm_dataframe

,Ticker,Price,Number of shares to buy,one-year price return,one-year return percentile,six-month price return,six-month return percentile,three-month price return,three-month return percentile,one-month price return,one-month return percentile,hqm_score
0,A,157.59,N/A,0.010338,63.073852,0.243408,88.622754,0.175555,90.419162,0.197939,86.027944,N/A
1,AAL,14.68,N/A,-0.267686,19.760479,-0.141140,14.570858,0.043814,59.481038,0.053156,23.153693,N/A
2,AAP,154.86,N/A,-0.352359,12.774451,-0.165201,11.976048,-0.241150,2.59481,-0.127095,2.195609,N/A
3,AAPL,152.55,N/A,-0.060814,49.9002,0.059240,54.291417,-0.099102,17.56487,0.027899,17.165669,N/A
4,ABBV,162.97,N/A,0.412061,93.013972,0.101334,65.868263,0.160848,87.62475,0.086984,39.321357,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,127.45,N/A,0.018128,64.471058,0.123818,70.05988,0.110198,77.644711,0.145274,66.666667,N/A
497,ZBH,118.35,N/A,-0.093202,45.508982,-0.019494,34.93014,0.082719,71.057884,0.100891,44.311377,N/A
498,ZBRA,273.17,N/A,-0.574901,1.397206,-0.197160,8.982036,-0.151993,8.183633,0.030305,17.964072,N/A
499,ZION,53.69,N/A,-0.222786,24.750499,-0.037771,31.536926,-0.078719,22.355289,0.086925,39.121756,N/A


In [11]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentile = []
    for time in time_periods:
        momentum_percentile.append(hqm_dataframe.loc[row, f'{time} return percentile'])
    hqm_dataframe.loc[row, 'hqm_score'] = mean(momentum_percentile)
    
# sorting the dataframe by HQM_score
#filtering the top 50 stocks

hqm_dataframe.sort_values(by='hqm_score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe

,Ticker,Price,Number of shares to buy,one-year price return,one-year return percentile,six-month price return,six-month return percentile,three-month price return,three-month return percentile,one-month price return,one-month return percentile,hqm_score
0,GILD,86.49,N/A,0.280027,88.423154,0.375282,96.407186,0.356624,98.802395,0.262506,95.608782,94.810379
1,ABMD,381.23,N/A,0.155144,79.241517,0.535880,99.600798,0.454529,99.401198,0.474790,100.0,94.560878
2,COG,22.36,N/A,0.347402,91.017964,0.187561,81.836327,0.265256,97.205589,0.258013,95.209581,91.317365
3,ROST,119.47,N/A,0.019199,64.670659,0.485568,99.001996,0.334227,98.203593,0.371294,99.001996,90.219561
4,TJX,83.88,N/A,0.159497,79.640719,0.354254,95.608782,0.277809,97.60479,0.198157,86.227545,89.770459
5,AFL,74.00,N/A,0.306370,89.620758,0.292000,92.015968,0.179149,90.618762,0.193784,84.630739,89.221557
6,AMP,338.40,N/A,0.115893,76.247505,0.286293,91.616766,0.209924,94.011976,0.251898,94.610778,89.121756
7,PFG,93.74,N/A,0.298772,89.221557,0.338831,95.209581,0.219486,94.810379,0.168913,76.247505,88.872255
8,APD,317.46,N/A,0.058073,70.459082,0.324104,94.411178,0.193391,92.215569,0.289664,97.205589,88.572854
9,FTI,12.65,N/A,0.938733,99.201597,0.518162,99.201597,0.429075,99.201597,0.126241,56.487026,88.522954


In [12]:
portfolio = float(portfolio_size)/len(hqm_dataframe.index)

for name in range(len(hqm_dataframe.index)):
    hqm_dataframe.loc[name, 'Number of shares to buy'] = math.floor(portfolio/hqm_dataframe.loc[name, 'Price'])
    
hqm_dataframe

,Ticker,Price,Number of shares to buy,one-year price return,one-year return percentile,six-month price return,six-month return percentile,three-month price return,three-month return percentile,one-month price return,one-month return percentile,hqm_score
0,GILD,86.49,2312,0.280027,88.423154,0.375282,96.407186,0.356624,98.802395,0.262506,95.608782,94.810379
1,ABMD,381.23,524,0.155144,79.241517,0.535880,99.600798,0.454529,99.401198,0.474790,100.0,94.560878
2,COG,22.36,8944,0.347402,91.017964,0.187561,81.836327,0.265256,97.205589,0.258013,95.209581,91.317365
3,ROST,119.47,1674,0.019199,64.670659,0.485568,99.001996,0.334227,98.203593,0.371294,99.001996,90.219561
4,TJX,83.88,2384,0.159497,79.640719,0.354254,95.608782,0.277809,97.60479,0.198157,86.227545,89.770459
5,AFL,74.00,2702,0.306370,89.620758,0.292000,92.015968,0.179149,90.618762,0.193784,84.630739,89.221557
6,AMP,338.40,591,0.115893,76.247505,0.286293,91.616766,0.209924,94.011976,0.251898,94.610778,89.121756
7,PFG,93.74,2133,0.298772,89.221557,0.338831,95.209581,0.219486,94.810379,0.168913,76.247505,88.872255
8,APD,317.46,630,0.058073,70.459082,0.324104,94.411178,0.193391,92.215569,0.289664,97.205589,88.572854
9,FTI,12.65,15810,0.938733,99.201597,0.518162,99.201597,0.429075,99.201597,0.126241,56.487026,88.522954


In [14]:
hqm_dataframe.to_excel('momentum.xlsx')